In [105]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib. pyplot as plt
import matplotlib
from sklearn.preprocessing import MinMaxScaler
from keras. layers import LSTM, Dense, Dropout
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib. dates as mandates
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from keras.models import Sequential
from keras.layers import Dense
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras import *

In [106]:
data = pd.read_csv('../input/stock-bse/BSE_30.csv')


In [107]:
nan_value = float("NaN")
data.replace(" ", nan_value, inplace=True)

data.dropna(subset = ["Symbol","Date","Open","High","Low", "Adj Close", "Adj Close"], inplace=True)

In [108]:
data.isnull().values.any()

In [109]:
all_stock_names = data['Symbol'].unique()
print(all_stock_names)

In [110]:
 # 1. Getting a stock name
stock_name = input("Enter a Stock Price Name: ")
 
# 2. Extrating all the data having the name same as the stock name entered
data_name = data['Symbol'] == stock_name
 
# 3. Putting all the rows of specific stock in a variable
final_data = data[data_name]
 
# 4. Printing first 5 rows of the stock data of a specific stock name
final_data.head()
data=final_data

In [111]:
data["Adj Close"].plot()

In [112]:
output_var = pd.DataFrame(data["Adj Close"])
#Selecting the Features
features = ["Open", "High", "Low", "Volume"]

In [113]:
scaler = MinMaxScaler()
feature_transform = scaler.fit_transform(data[features])
feature_transform= pd.DataFrame(columns=features, data=feature_transform, index=data.index)
feature_transform.head()

In [114]:
timesplit= TimeSeriesSplit(n_splits=10)
for train_index, test_index in timesplit.split(feature_transform):
        X_train, X_test = feature_transform[:len(train_index)], feature_transform[len(train_index): (len(train_index)+len(test_index))]
        y_train, y_test = output_var[:len(train_index)].values.ravel(), output_var[len(train_index): (len(train_index)+len(test_index))].values.ravel()

In [115]:
trainX =np.array(X_train)
testX =np.array(X_test)
X_train = trainX.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = testX.reshape(X_test.shape[0], 1, X_test.shape[1])

In [116]:
lstm = Sequential()
lstm.add(LSTM(32, input_shape=(1, trainX.shape[1]), activation="relu", return_sequences=False))
lstm.add(Dense(1))
lstm.compile(loss="mean_squared_error", optimizer="adam")
#plot_model(lstm, show_shapes=True, show_layer_names=True)

In [117]:
history=lstm.fit(X_train, y_train, epochs=100, batch_size=8, verbose=1, shuffle=False)


In [118]:
y_pred= lstm.predict(X_test)


In [119]:
plt.plot(y_test, label="True Value")
plt.plot(y_pred, label="LSTM Value")
plt.title("Prediction by LSTM")
plt.xlabel("Time Scale")
plt.ylabel("Scaled USD")
plt.legend()
plt.show()